In [3]:
import matplotlib.pyplot as plt
import pyGMs as gm
import numpy as np
import torch
import pyro
import pyro.infer
import pyro.optim
import pyro.distributions as dist
import pyro.poutine as poutine

import pandas as pd

seed = 123
pyro.set_rng_seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [4]:
data = pd.read_csv('data.csv')

data = data[["winner", "loser"]]
data["win"] = 1.0

dataset = list(data.itertuples(index=False, name=None))

teams = set()
for match in dataset:
    teams.add(match[0])
    teams.add(match[1])

teams = {v: i for i, v in enumerate(list(teams))}
dataset = [(teams[winner], teams[loser], win)
           for winner, loser, win in dataset]

In [6]:
def model(matches, n_teams):
    X = [pyro.sample(f"X{i}", dist.Normal(0, 1)) for i in range(n_teams)]

    for i, m in enumerate(matches):
        pyro.sample(f"w{i}",
                    dist.Bernoulli(logits=X[m[0]]-X[m[1]]),
                    obs=torch.tensor(1.))

In [ ]:
nuts_kernel = pyro.infer.NUTS(model, jit_compile=False)
mcmc = pyro.infer.MCMC(nuts_kernel, num_samples=50,
                       warmup_steps=10, num_chains=1)
mcmc.run(dataset, len(teams))